In [1]:
!git clone https://github.com/felixkreuk/SegFeat

Cloning into 'SegFeat'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 55 (delta 14), reused 12 (delta 12), pack-reused 34 (from 1)
Receiving objects: 100% (55/55), 32.80 MiB | 38.30 MiB/s, done.
Resolving deltas: 100% (23/23), done.


In [7]:
%cd /kaggle/working

/kaggle/working


In [8]:
%%writefile segfeat.patch
--- dataloader.py.orig	2026-01-30 19:00:26
+++ dataloader.py	2026-01-30 19:00:56
@@ -208,7 +208,10 @@
         raise NotImplementedError
 
     def process_file(self, wav_path):
-        phn_path = wav_path.replace("wav", "phn")
+        base = wav_path.rsplit(".", 1)[0]
+        phn_path = base + ".phn"
+        if not os.path.exists(phn_path):
+            phn_path = base + ".PHN"
 
         # load audio
         spect = extract_features(wav_path, self.hparams)
@@ -235,7 +238,7 @@
 
     def _make_dataset(self):
         files = []
-        wavs = list(iter_find_files(self.wav_path, "*.wav"))
+        wavs = list(iter_find_files(self.wav_path, "*.wav")) + list(iter_find_files(self.wav_path, "*.WAV"))
         if self.hparams.devrun:
             wavs = wavs[:self.hparams.devrun_size]
 
@@ -265,10 +268,19 @@
         self.data = self._make_dataset()
 
     @staticmethod
-    def get_datasets(hparams):
-        train_dataset = TimitDataset(join(hparams.wav_path, 'train'),
+    def _find_subdir(base, name):
+        """Find a subdirectory case-insensitively."""
+        target = name.lower()
+        for entry in os.listdir(base):
+            if entry.lower() == target and os.path.isdir(join(base, entry)):
+                return join(base, entry)
+        return join(base, name)
+
+    @staticmethod
+    def get_datasets(hparams):
+        train_dataset = TimitDataset(TimitDataset._find_subdir(hparams.wav_path, 'train'),
                                      hparams)
-        test_dataset  = TimitDataset(join(hparams.wav_path, 'test'),
+        test_dataset  = TimitDataset(TimitDataset._find_subdir(hparams.wav_path, 'test'),
                                      hparams)
 
         train_len   = len(train_dataset)


Overwriting segfeat.patch


In [9]:
%cd SegFeat

/kaggle/working/SegFeat


In [10]:
!git apply ../segfeat.patch


In [12]:
%%writefile requirements.txt
torch
torchaudio
torchvision
pytorch-lightning
boltons
loguru
librosa
numpy
pandas
soundfile
tqdm

Writing requirements.txt


In [13]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.2/194.2 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 3.3 MB/s eta 0:00:00


In [14]:
!python main.py --wav_path /kaggle/input/darpa-timit-acousticphonetic-continuous-speech/data --dataset timit --delta_feats --dist_feats

Traceback (most recent call last):
  File "/kaggle/working/SegFeat/main.py", line 14, in <module>
    from pytorch_lightning.logging import TestTubeLogger
ModuleNotFoundError: No module named 'pytorch_lightning.logging'
